In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Dense, Activation, Lambda, Input, Concatenate, Multiply
from tensorflow.keras.metrics import binary_accuracy
from tensorflow.keras.losses import binary_crossentropy
from tensorflow.keras import backend as K
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from tensorflow.keras import callbacks
tf.compat.v1.disable_eager_execution()

file_path = "../DataSet/EXAMPLE_DATASET/result_to_neural_250.txt"  # Dosya yolunu buraya göre ayarlayın



In [ ]:
def read_input_file(file_path):
    weights = []
    values = []
    capacities = []
    best_picks = []

    with open(file_path, 'r') as file:
        lines = file.readlines()
        for i in range(0, len(lines), 4):
            # Okunan satırları boşluklarla ayrılan değerlere ayırarak diziye atama
            value_line  = list(map(int, lines[i].split()))
            weight_line = list(map(int, lines[i + 1].split()))
            capacity_line = list(map(int, lines[i + 2].split()))
            best_picks_line = list(map(int, lines[i + 3].split()))
            

            weights.append(weight_line)
            values.append(value_line)
            capacities.append(capacity_line)
            best_picks.append(best_picks_line)

    return weights, values, capacities, best_picks

def create_knapsack_dataset(file_path, count,i):
    weights, values, capacities, best_picks = read_input_file(file_path)
    x = [[],[]]
    y = [[]]
    max_price = 0
    max_weight = 0
  
    for i in range(count):
        item_weights = weights[i]
        item_values = values[i]
        item_capacity = capacities[i]
        item_count = len(item_weights)

        # Max price ve max weight değerlerini güncelleme
        max_price = max(max_price, np.max(item_values))
        max_weight = max(max_weight, np.max(item_weights))


        # Normalize prices ve weights hesaplama
        normalized_prices = [price / max_price for price in item_values]
        normalized_weights = [weight / max_weight for weight in item_weights]

        # Capacity değerini normalize etme
        normalized_capacity = item_capacity / max_weight
        


        x[0].append(normalized_weights)
        x[1].append(normalized_prices)
        #x[2].append(normalized_capacity)
        y[0].append(best_picks[i])

    return x, y

In [ ]:

X_train, y_train = create_knapsack_dataset(file_path,9800,0)
X_test, y_test = create_knapsack_dataset(file_path,200,9801)



In [ ]:
def metric_overprice(input_prices):
    def overpricing(y_true, y_pred):
        y_pred = K.round(y_pred)
        return K.mean(K.batch_dot(y_pred, input_prices, 1) - K.batch_dot(y_true, input_prices, 1))

    return overpricing
    
def metric_space_violation(input_weights):
    def space_violation(y_true, y_pred):
        y_pred = K.round(y_pred)
        return K.mean(K.maximum(K.batch_dot(y_pred, input_weights, 1) - 1, 0))
    return space_violation


In [ ]:
earlystopping = callbacks.EarlyStopping(monitor="loss",
                                        mode="min", patience=5,
                                        restore_best_weights=True)

def train_knapsack(model,save_weights = False,model_name="first" ):
    from keras.callbacks import ModelCheckpoint
    import os
    if save_weights:
        model.fit(X_train, y_train, epochs=100, verbose=0, callbacks=[ModelCheckpoint(model_name+".h5", monitor="loss", save_best_only=True, save_weights_only=True),earlystopping])
    else :
        model.load_weights("/content/" + model_name + ".h5")
    train_results = model.evaluate(X_train, y_train, 100, 0)
    test_results = model.evaluate(X_test, y_test, 100, 0);
    print("Model results(Train/Test):")
    print(f"Loss:               {train_results[0]:.2f} / {test_results[0]:.2f}")
    print(f"Binary accuracy:    {train_results[1]:.2f} / {test_results[1]:.2f}")
    print(f"Space violation:    {train_results[2]:.2f} / {test_results[2]:.2f}")
    print(f"Overpricing:        {train_results[3]:.2f} / {test_results[3]:.2f}")

In [ ]:
def supervised_continues_knapsack(item_count=5):
    input_shape = (item_count,)
    input_weights = Input(shape=input_shape)
    input_prices = Input(shape=input_shape)
    inputs_concat = Concatenate()([input_weights, input_prices])
    picks = Dense(item_count, use_bias=False, activation="sigmoid")(inputs_concat)
    model = Model(inputs=[input_weights, input_prices], outputs=[picks])
    model.compile("adam",
                  binary_crossentropy,
                  metrics=[binary_accuracy, metric_space_violation(input_weights),
                           metric_overprice(input_prices)])
    return model
model = supervised_continues_knapsack(250)
train_knapsack(model,True,"first")


In [ ]:
def supervised_continues_knapsack_one_hidden(item_count=5):
    input_weights = Input((item_count,))
    input_prices = Input((item_count,))
    input_capacity = Input((1,))
    inputs_concat = Concatenate()([input_weights, input_prices])
    picks = Dense(item_count * 10, use_bias=False, activation="sigmoid")(inputs_concat)
    picks = Dense(item_count, use_bias=False, activation="sigmoid")(picks)
    model = Model(inputs=[input_weights, input_prices], outputs=[picks])
    model.compile("adam",
                  binary_crossentropy,
                  metrics=[binary_accuracy, metric_space_violation(input_weights),
                           metric_overprice(input_prices)])
    return model
model = supervised_continues_knapsack_one_hidden(250)
train_knapsack(model,False,"best_model")

In [ ]:
def supervised_discrete_knapsack(item_count=5):
    input_weights = Input((item_count,))
    input_prices = Input((item_count,))
    input_capacity = Input((1,))
    inputs_concat = Concatenate()([input_weights, input_prices])
    concat_tanh = Dense(item_count, use_bias=False, activation="tanh")(inputs_concat)
    concat_sigmoid = Dense(item_count, use_bias=False, activation="sigmoid")(inputs_concat)
    concat_multiply = Multiply()([concat_sigmoid, concat_tanh])
    picks = Multiply()([concat_multiply, concat_multiply])
    model = Model(inputs=[input_weights, input_prices], outputs=[picks])
    model.compile("sgd",
                  binary_crossentropy,
                  metrics=[binary_accuracy, metric_space_violation(input_weights),
                           metric_overprice(input_prices)])
    return model

model = supervised_discrete_knapsack(250)
train_knapsack(model)

In [ ]:

from kerastuner.tuners import RandomSearch


def build_model(hp,item_count = 250):
    input_shape = (item_count,)
    input_weights = Input(shape=input_shape)
    input_prices = Input(shape=input_shape)
    inputs_concat = Concatenate()([input_weights, input_prices])
    
    hp_units = hp.Int('units', min_value=32, max_value=128, step=32)
    hp_learning_rate = hp.Choice('learning_rate', values=[0.001, 0.01, 0.1], default=0.01)

    dense_1 = Dense(hp_units, activation='relu')(inputs_concat)
    picks = Dense(item_count, use_bias=False, activation="sigmoid")(dense_1)
    
    model = Model(inputs=[input_weights, input_prices], outputs=[picks])
    
    model.compile(optimizer=tf.keras.optimizers.legacy.Adam(learning_rate=hp_learning_rate),
              loss=binary_crossentropy,
              metrics=[binary_accuracy, metric_space_violation(input_weights),
                       metric_overprice(input_prices)])

    return model

tuner = RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=5,
    executions_per_trial=3,
    directory='tuner_directory',
    project_name='knapsack_tuner'
)

tuner.search(X_train, y_train, epochs=50, validation_data=(X_test, y_test), batch_size=64, verbose=1)

best_model = tuner.get_best_models(num_models=1)[0]
best_model.summary()

best_model.fit(X_train, y_train, epochs=50, batch_size=64, verbose=1)

train_results = best_model.evaluate(X_train, y_train, batch_size=64)
test_results = best_model.evaluate(X_test, y_test, batch_size=64)

print("Model results (Train/Test):")
print(f"Loss:               {train_results[0]:.2f} / {test_results[0]:.2f}")
print(f"Binary accuracy:    {train_results[1]:.2f} / {test_results[1]:.2f}")
print(f"Space violation:    {train_results[2]:.2f} / {test_results[2]:.2f}")
print(f"Overpricing:        {train_results[3]:.2f} / {test_results[3]:.2f}")
